# Introduction



A hypothethical real estate owner and operator is looking to identify potential retail tenants to the ground retail space of various office and multfaimily buildings in Manhattan. The company would like to avoid using a leasing agent as they can be expensive and the firm believes they can use the new data science skills learned by their analyst to identify potential tenants. The company is hoping to identify which types of businesses it should target based on neighborhood in Manhattan. If the project is successful in identifying popular businesses in each neighborhood of Manhattan, the company will save money on a leasing agent and decrease the time it takes to rent the vacant space. 

# Data

* This project will use two datasets, FourSquare API and NYU Neighborhood Names data. 
    * Foursquare API
        * The Foursquare API will be used to identify popular venues by neighborhood in Manhattan based on ratings, tips, and likes. 
    * 2014 New York City Neighborhood Names
        * The NYC Neighborhood Data, which comes as a .json file, will be used to cluster businesses from the Foursquare API by neighborhood. 

# Methodology

First, I replicated the labs that we used throughout the course to load data from the data sets and clean the data. I imported data from the NYC Neighborhood Names data base and extracted the necessary fields: Borough, Neighborhood, Latitude, and Longitude. The data was then refined to only include the borough of Manhattan. These were placed into a pandas dataframe to be later used with the FourSqaure data.


Next, I gathered information using the Foursquare API on venue name, category, latitude and longitude. The data was then put into a pandas dataframe and venue frequency was assessed by neighborhood. Then onehot encoding was used to translate categorical variables into binary vectors. Once this was done, we assigned a frequency to each venue type and created a new dataframe with neighborhoods and their top 10 venues. 

# Results

## Below is a list of the top 10 venues by neighborhood in Manhattan:

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site,Boat or Ferry,Italian Restaurant,Pizza Place,Playground,Plaza
1,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Pizza Place,Bakery,Bookstore,Cosmetics Shop,Gym,Gym / Fitness Center,Japanese Restaurant
2,Central Harlem,Gym / Fitness Center,American Restaurant,Seafood Restaurant,Chinese Restaurant,Bar,African Restaurant,Fried Chicken Joint,French Restaurant,Event Space,Library
3,Chelsea,Coffee Shop,Art Gallery,American Restaurant,Italian Restaurant,Bakery,Cocktail Bar,Pizza Place,Pet Store,Nightclub,Market
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Bakery,Dessert Shop,Spa,Hotpot Restaurant,Vietnamese Restaurant,Optical Shop,Salon / Barbershop
5,Civic Center,Coffee Shop,Cocktail Bar,French Restaurant,Hotel,Yoga Studio,Sushi Restaurant,American Restaurant,Café,Gym / Fitness Center,Park
6,Clinton,Theater,Gym / Fitness Center,Coffee Shop,Italian Restaurant,Spa,Wine Shop,Gym,Hotel,American Restaurant,Thai Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Latin American Restaurant,French Restaurant,Dance Studio,Beer Bar,Taco Place,Gas Station
8,East Village,Bar,Pizza Place,Coffee Shop,Mexican Restaurant,Wine Bar,Ice Cream Shop,Ramen Restaurant,Speakeasy,Korean Restaurant,Cocktail Bar
9,Financial District,Coffee Shop,Pizza Place,Mexican Restaurant,Hotel,Cocktail Bar,Sandwich Place,Juice Bar,American Restaurant,Gym,Park


# Discussion

As we can see above, each neighborhood in Manhattan has the top 10 venues identified. Our hypothetical real estate company could use this matrix to identify the companies to target by the neighborhood. In addition, the data that we obtained could be used to determine which business type is overcrowded and should not be pursued. While this analysis did not go into this level of detail, Foursquare can pull specific data about the tenant. For example, we can pull data that shows that Subway restaurants are in every neighborhood but the West Village, which could be the next location for a new store. 

# Conclusion

As we can see, using readily available data and simple data analysis techniques taught in these courses, we can address a somewhat challenging problem. The hypothethical real estate company could easily look at the matrix above and determine what is the appropriate business to pursue. This would save them both time and money. 

# Notebook and Analysis Below

## Download Data

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [6]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data loaded!')

Data loaded!


In [7]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [11]:
#define variable that contains all the necessary information
neighborhoods_data = newyork_data['features']

In [12]:
#create pandas dataframe 
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)

In [52]:
#loop data into dataframe
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [53]:
manhattan_neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [24]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: MCY2TQYTUH45POO2HEU5GPJ4JX0OGBG50B24KRTZ2XUVFSN2
CLIENT_SECRET:KLBOKAKHT0XMZTPNWINDLGWGEEAG3HW1MYKJHHVPRC0LH2XX


In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
manhattan_venues = getNearbyVenues(names=manhattan_neighborhoods['Neighborhood'],
                                   latitudes=manhattan_neighborhoods['Latitude'],
                                   longitudes=manhattan_neighborhoods['Longitude']
                                  )


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyve

In [58]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,140,140,140,140,140,140
Carnegie Hill,176,176,176,176,176,176
Central Harlem,90,90,90,90,90,90
Chelsea,200,200,200,200,200,200
Chinatown,200,200,200,200,200,200
Civic Center,200,200,200,200,200,200
Clinton,200,200,200,200,200,200
East Harlem,78,78,78,78,78,78
East Village,200,200,200,200,200,200


In [ ]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot

In [36]:
manhattan_onehot.shape

(6322, 324)

In [37]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaur

In [38]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.09
1          Hotel  0.07
2            Gym  0.06
3  Boat or Ferry  0.04
4  Memorial Site  0.04


----Carnegie Hill----
                venue  freq
0         Coffee Shop  0.08
1                Café  0.06
2  Italian Restaurant  0.03
3              Bakery  0.03
4           Wine Shop  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.04
1   American Restaurant  0.04
2    Chinese Restaurant  0.04
3   Fried Chicken Joint  0.04
4  Gym / Fitness Center  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.09
1          Art Gallery  0.05
2  American Restaurant  0.04
3   Italian Restaurant  0.03
4               Bakery  0.03


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.09
1         Cocktail Bar  0.05
2  American Restaurant  0.04
3               Bakery  0.04
4                  Spa  0.03


----Civic Center----
               venue  freq
0  

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site,Boat or Ferry,Italian Restaurant,Pizza Place,Playground,Plaza
1,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Pizza Place,Bakery,Bookstore,Cosmetics Shop,Gym,Gym / Fitness Center,Japanese Restaurant
2,Central Harlem,Gym / Fitness Center,American Restaurant,Seafood Restaurant,Chinese Restaurant,Bar,African Restaurant,Fried Chicken Joint,French Restaurant,Event Space,Library
3,Chelsea,Coffee Shop,Art Gallery,American Restaurant,Italian Restaurant,Bakery,Cocktail Bar,Pizza Place,Pet Store,Nightclub,Market
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Bakery,Dessert Shop,Spa,Hotpot Restaurant,Vietnamese Restaurant,Optical Shop,Salon / Barbershop
5,Civic Center,Coffee Shop,Cocktail Bar,French Restaurant,Hotel,Yoga Studio,Sushi Restaurant,American Restaurant,Café,Gym / Fitness Center,Park
6,Clinton,Theater,Gym / Fitness Center,Coffee Shop,Italian Restaurant,Spa,Wine Shop,Gym,Hotel,American Restaurant,Thai Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Latin American Restaurant,French Restaurant,Dance Studio,Beer Bar,Taco Place,Gas Station
8,East Village,Bar,Pizza Place,Coffee Shop,Mexican Restaurant,Wine Bar,Ice Cream Shop,Ramen Restaurant,Speakeasy,Korean Restaurant,Cocktail Bar
9,Financial District,Coffee Shop,Pizza Place,Mexican Restaurant,Hotel,Cocktail Bar,Sandwich Place,Juice Bar,American Restaurant,Gym,Park
